In [1]:
import sys
sys.path.append('../')
import wrangle

import nlp

In [22]:
from sklearn.decomposition import LatentDirichletAllocation, NMF

In [75]:
from sklearn.feature_extraction.text import CountVectorizer

In [38]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as esw

stopWords = ['yes', 'sort', 've', 'thing', 'try', '50', 'conference'] + list(esw) 

## SPACING

In [2]:
df, data_dict = wrangle.wrangle_data()

In [15]:
execs = nlp.create_persona_corpus(df, 5)

In [16]:
execs.big_answer.apply(nlp.find_subjectivity).mean()

0.4378043751465303

In [17]:
nlp.show_persona_keywords(execs.big_answer, 0.8, 3, (1,3), 15)

['travel',
 'ticket',
 'insight',
 'attendeesmultitrack',
 'chooses',
 'path',
 'multiple',
 'option',
 'available',
 'timeslotsnananana',
 'attendeesmultitrack attendee',
 'attendee chooses',
 'chooses path',
 'path multiple',
 'multiple option']

## Creating topic feature

In [18]:
input_column = df.recommendations

In [19]:
doc_term_matrix, count_vect = nlp.create_wordcount_matrix(input_column, ngram=(1,3))

In [23]:
LDA = LatentDirichletAllocation(n_components=3, random_state=42)
LDA.fit(doc_term_matrix)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=3, n_jobs=None,
                          perp_tol=0.1, random_state=42, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)

In [ ]:
LDA.fit(matrix).transform(matrix)

In [28]:
LDA.transform(doc_term_matrix)

array([[0.07695014, 0.85603253, 0.06701733],
       [0.90353362, 0.05253518, 0.04393121],
       [0.09234123, 0.81580031, 0.09185845],
       ...,
       [0.98606397, 0.00712634, 0.00680968],
       [0.95229316, 0.02419899, 0.02350784],
       [0.16666809, 0.1666694 , 0.66666251]])

In [29]:
lda_H = LDA.transform(doc_term_matrix)

In [31]:
input_column

resp_id
284                        research-practice connections
288    Don’t create schedule conflicts with other rel...
294                                  Looking forward! \n
281                                                  NaN
280                                                  NaN
                             ...                        
869    provide tools and resources and support for ne...
879          Make it affordable. Somewhere in Europe. :)
136    My goal is to provide constructive criticism, ...
74     Instead of trying to create a general ideal co...
2                                                    NaN
Name: recommendations, Length: 726, dtype: object

In [30]:
lda_H.shape

(726, 3)

In [32]:
lda_H[0]

array([0.07695014, 0.85603253, 0.06701733])

In [62]:
np.amax(lda_H, axis=0)

array([0.99425199, 0.98970397, 0.98605021])

In [56]:
np.argsort(lda_H[0]).min()

0

In [41]:
np.sort(lda_H[0])

array([0.06701733, 0.07695014, 0.85603253])

In [64]:
topic_doc_series = pd.Series(lda_H)

Exception: Data must be 1-dimensional

In [46]:
topic_doc_df.iloc[0]

0    0.076950
1    0.856033
2    0.067017
Name: 0, dtype: float64

In [72]:
topic_doc_df.idxmax(axis=1)

0      1
1      0
2      1
3      2
4      2
      ..
721    1
722    2
723    0
724    0
725    2
Length: 726, dtype: int64

In [25]:
len(LDA.components_[0])

1452

In [73]:
def assign_topic(input_column):
    count_vect = CountVectorizer(max_df=.8, min_df=2, stop_words='english', ngram_range=(1,3))
    doc_term_matrix = count_vect.fit_transform(input_column.values.astype('U'))
    LDA = LatentDirichletAllocation(n_components=3, random_state=42)
    LDA.fit(doc_term_matrix)
    lda_H = LDA.transform(doc_term_matrix)
    topic_doc_df = pd.DataFrame(lda_H)
    return topic_doc_df.idxmax(axis=1)

In [76]:
assign_topic(input_column)

0      1
1      0
2      1
3      2
4      2
      ..
721    1
722    2
723    0
724    0
725    2
Length: 726, dtype: int64